<a href="https://colab.research.google.com/github/flaviowu/btc-c14-g4/blob/main/btc-c14-g4/notebooks/NLP_empresas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Leitura dos Dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/BTC-Dados-G4/train_clean_rev2.csv')
test = pd.read_csv('/content/drive/MyDrive/BTC-Dados-G4/train_clean_rev2.csv')

### Drop Linhas Dúplicadas

In [ ]:
dados_unicos.shape

(1341267, 11)

In [ ]:
def Apaga_Duplicado(data='x', colunas=['y']):
  data = data.drop_duplicates(subset = colunas, keep= 'first', inplace = False)
  return data

In [ ]:
dados_unicos = Apaga_Duplicado(dados_unicos, ['name','brand_name','price'])

In [ ]:
dados_unicos.shape

(1308675, 11)

### Separação de dados em df Marcas nulas e preenchidas. Limpeza da coluna Descrição

In [ ]:
dados_unicos.drop(['date', 'stock'], axis=1, inplace=True)

In [ ]:
DF_sem_marca = dados_unicos.loc[train['brand_name'].isnull() == True]

In [ ]:
DF_com_marca = dados_unicos.drop(DF_sem_marca.index, axis=0)

In [ ]:
DF_com_marca.shape, DF_sem_marca.shape

((733973, 9), (574702, 9))

### Dividir para conquistar

In [ ]:
def transforme_nlp(serie, data):
  data_series = serie.apply( lambda x: nlp(x) )
  data['empresa_encontrada'] = data_series.apply( lambda x: [ent for ent in x.ents if ent.label_ == 'ORG'])

In [ ]:
amostra_1 = DF_sem_marca['item_description'][0:30000]
amostra_2 = DF_sem_marca['item_description'][30001:60000]
amostra_3 = DF_sem_marca['item_description'][60001:90000]
amostra_4 = DF_sem_marca['item_description'][90001:120000]
amostra_5 = DF_sem_marca['item_description'][120001:150000]
amostra_6 = DF_sem_marca['item_description'][150001:180000]
amostra_7 = DF_sem_marca['item_description'][180001:210000]
amostra_8 = DF_sem_marca['item_description'][210001:240000]
amostra_9 = DF_sem_marca['item_description'][240001:270000]

In [ ]:
amostra_10 = DF_sem_marca['item_description'][270001:300000]
amostra_11 = DF_sem_marca['item_description'][300001:330000]
amostra_12 = DF_sem_marca['item_description'][330001:360000]
amostra_13 = DF_sem_marca['item_description'][360001:390000]
amostra_14 = DF_sem_marca['item_description'][390001:420000]
amostra_15 = DF_sem_marca['item_description'][420001:450000]
amostra_16 = DF_sem_marca['item_description'][450001:480000]
amostra_17 = DF_sem_marca['item_description'][480001:520000]
amostra_18 = DF_sem_marca['item_description'][520000:]

In [ ]:
lista = [ amostra_1, amostra_2, amostra_3, amostra_4, amostra_5, amostra_6, amostra_7, amostra_8, amostra_9,
        amostra_10, amostra_11, amostra_12, amostra_13, amostra_14, amostra_15, amostra_16, amostra_17, amostra_18]

In [ ]:
for amostra in range(len(lista)):
  transforme_nlp(lista[amostra], DF_sem_marca)
  DF_sem_marca.to_csv(f"Amostra {amostra}", encoding = 'utf-8')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Unindo as amostras

In [ ]:
lista = []
for posicao in range(18):
  dados_unificados_sm = pd.read_csv(f'/content/Amostra {posicao}')
  apagar = dados_unificados_sm.loc[dados_unificados_sm['empresa_encontrada'] == "[]"]
  dados_unificados_sm.drop(apagar.index, axis=0, inplace=True)
  lista.append(dados_unificados_sm)
  if posicao == 0:
    cont_empresa = dados_unificados_sm
  else:
    cont_empresa = pd.concat([cont_empresa, lista[posicao]], ignore_index=True)
    cont_empresa = Apaga_Duplicado(cont_empresa, ['empresa_encontrada', 'name', 'price'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
cont_empresa.shape

(626426, 11)

In [ ]:
cont_empresa.to_csv(f"unificado", encoding = 'utf-8')

## Conjuntos de Treino

São três conjuntos de Treino:

1 - Com linhas de Marcas preenchidas;

2 - Sem a coluna de Marca;

3 - Marcas a partir da NPL onde as linhas estavam vazias.



In [ ]:
DF_com_marca.shape # primeiro conjunto de treino

(733973, 9)

In [ ]:
train.drop('brand_name', 'date', 'stock'] axis=1, inplace=True)

In [ ]:
train.shape # Segundo conjunto de treino

(1476204, 8)

In [ ]:
cont_empresa = cont_empresa.reindex(columns= ['name', 'item_condition_id', 'empresa_encontrada', 'price',
       'shipping', 'item_description', 'main_cat', 'sub_cat_1', 'sub_cat_2',
       'brand_name'])

In [ ]:
cont_empresa.drop('brand_name', axis=1, inplace=True)

In [ ]:
cont_empresa.columns = ['name', 'item_condition_id', 'brand_name', 'price', 'shipping',
       'item_description', 'main_cat', 'sub_cat_1', 'sub_cat_2']

In [ ]:
DF_unificado = pd.concat([ DF_com_marca, cont_empresa ], ignore_index=True)

In [ ]:
DF_unificado = DF_unificado.dropna()

In [ ]:
DF_unificado['brand_name'] = DF_unificado['brand_name'].apply(lambda x: str(x))

In [ ]:
DF_unificado['brand_name'] = DF_unificado['brand_name'].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", '', x))

In [ ]:
DF_unificado.to_csv(f"DF_empresas_nlp", encoding = 'utf-8')

In [ ]:
DF_unificado.shape  # terceiro conjunto de treino

(785697, 9)